In [ ]:
access_token = ""

In [286]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback, IntervalStrategy
#from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score # Import all needed metrics

In [287]:
id = 3
torch.cuda.empty_cache()
if torch.cuda.is_available():
    GPU = torch.cuda.get_device_name(id)
    print(f"PyTorch available GPU: {GPU}")
else:
    print("GPU not available")

PyTorch available GPU: NVIDIA RTX A5000


In [288]:
torch.cuda.device_count()

4

In [289]:
device = torch.device(f"cuda:{id}" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:3


In [290]:
# device = torch.device("cpu")
# print(f'Device: {device}')

In [291]:
dataset = pd.read_csv('/home/patrick.araujo/llama2/datasets/balanced_output.csv')

In [292]:
dataset

,level_0,index,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sentiment,lengthContent,Language
0,2812,4087,4087,7c7a7fe6-f187-48c6-823a-70ae734b04a3,"the app runs a little slow at times, but nothi...",5,5,26.23.0.100,2023-12-09 10:57:59,NaN,NaN,26.23.0.100,2,145,en
1,17483,24985,24985,c76ab0e8-50f0-48ec-a920-9de9329299fa,You do realize there is more than just apple S...,3,0,22.19.0.100,2022-05-06 03:05:51,NaN,NaN,22.19.0.100,1,379,en
2,7525,10956,10956,5cd981cf-c9be-45ee-b0e3-92af3d7c2a12,"Easy to use if you know what you want, simple ...",5,2,26.21.0.100,2023-11-01 11:15:01,NaN,NaN,26.21.0.100,2,208,en
3,7019,10263,10263,74daab50-f012-4d3d-a880-9a3f0e4fd159,Work ok but frozen on loading and searching as...,3,1,26.21.0.100,2023-11-05 12:08:47,NaN,NaN,26.21.0.100,1,52,en
4,8328,12432,12432,af3b1dfc-9982-49d2-ab96-d06f2b9ae9a6,easy shopping and fast delivery,5,0,26.19.4.100,2023-10-10 17:42:38,NaN,NaN,26.19.4.100,2,31,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11626,4226,6159,6159,6eb97318-ab71-4b6c-a7af-d486bc38953a,love it can always find what I need and great ...,5,0,26.20.0.100,2023-11-24 15:20:22,NaN,NaN,26.20.0.100,2,83,en
11627,9673,14414,14414,0e6be9a1-7922-431e-b202-2907cfbaae87,The app is SLOW and not intuitive at all! Sear...,2,0,26.15.0.100,2023-08-09 07:22:09,NaN,NaN,26.15.0.100,0,298,en
11628,3504,5093,5093,37341abd-e64f-4e4e-af5f-1148dfc23a31,Love Amazon if you're not happy with a purchas...,5,1,26.22.0.100,2023-12-01 21:41:25,NaN,NaN,26.22.0.100,2,221,en
11629,15987,22652,22652,6f9ccf0a-bb3d-4490-9b30-a1c5e0e62f18,Less cumbersomely awkward. Improved member con...,3,0,24.5.2.100,2022-08-04 06:18:49,NaN,NaN,24.5.2.100,1,63,en


In [293]:
dataset.shape

(11631, 15)

In [294]:
sentiment_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(sentiment_counts_balanced)

sentiment
0    3877
1    3877
2    3877
Name: count, dtype: int64


In [295]:
# Load the pre-trained BERT model and tokenizer
#model_name = "/home/patrick.araujo/llama2/llama/llama-2-7b-hf"
#model_name = "mistralai/Mistral-7B-v0.1"
#model_name = "bert-base-uncased"
#model_name = "google-t5/t5-base"
model_name = "google-t5/t5-small"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [296]:
# Move model to GPU
model = model.to(device)

In [297]:
# Create train and validation datasets
comments = dataset[['content', 'sentiment']]
train_data, val_data = train_test_split(comments, test_size=0.2, random_state=42)

In [257]:
useFileData = True

In [258]:
if useFileData:
    train_data = pd.read_csv('/home/patrick.araujo/topicModeling/train_data.csv')
    train_data = train_data[['content', 'sentiment']]
    val_data = pd.read_csv('/home/patrick.araujo/topicModeling/val_data.csv')
    val_data = train_data[['content', 'sentiment']]

In [259]:
train_data

,content,sentiment
0,"It functions, but as I am browsing products I ...",1
1,Good application but the Voice search doesn't ...,1
2,App is now incredibly laggy to the point I'm u...,0
3,I have gotten several nice items,2
4,Everytime I buy from Amazon it always cancle m...,0
...,...,...
9299,"Used to be fast, but had allowed down a lot in...",1
9300,We absolutely love Amazon! I honestly have fou...,2
9301,Of course ads are what you would expect from a...,1
9302,Love it! Amazon is just about the place to get...,2


In [260]:
# topics = pd.read_csv('/home/patrick.araujo/topicModeling/out.csv')
topics = pd.read_csv('/home/patrick.araujo/topicModeling/NewList.csv')

In [261]:
topics

,topic,sentiment
0,leave items plain,0
1,convenient shopping,2
2,pick customer,0
3,neighbors house,2
4,good experiences amazon,2
...,...,...
11597,option slow laggy,0
11598,amazing absolute shopping,2
11599,ordering getting delivered,2
11600,toyota sequoia,2


In [262]:
# topic_df = topicssentiment['topics', 'dominant_sentiment']]
topic_df = topics[['topic', 'sentiment']]
topic_df = topic_df.rename(columns={'topic': 'content', 'sentiment': 'sentiment'})
# topic_df = topic_df.rename(columns={'topic': 'content', 'sentiment': 'sentiment'}, inplace=True)

In [263]:
topic_df = topic_df.dropna(subset=topic_df.columns)

In [264]:
topic_df

,content,sentiment
0,leave items plain,0
1,convenient shopping,2
2,pick customer,0
3,neighbors house,2
4,good experiences amazon,2
...,...,...
11597,option slow laggy,0
11598,amazing absolute shopping,2
11599,ordering getting delivered,2
11600,toyota sequoia,2


In [265]:
train_data = pd.concat([train_data, topic_df])
# Now, reset the index of df
train_data = train_data.reset_index(drop=True)

In [266]:
train_data

,content,sentiment
0,"It functions, but as I am browsing products I ...",1
1,Good application but the Voice search doesn't ...,1
2,App is now incredibly laggy to the point I'm u...,0
3,I have gotten several nice items,2
4,Everytime I buy from Amazon it always cancle m...,0
...,...,...
20901,option slow laggy,0
20902,amazing absolute shopping,2
20903,ordering getting delivered,2
20904,toyota sequoia,2


In [267]:
val_data

,content,sentiment
0,"It functions, but as I am browsing products I ...",1
1,Good application but the Voice search doesn't ...,1
2,App is now incredibly laggy to the point I'm u...,0
3,I have gotten several nice items,2
4,Everytime I buy from Amazon it always cancle m...,0
...,...,...
9299,"Used to be fast, but had allowed down a lot in...",1
9300,We absolutely love Amazon! I honestly have fou...,2
9301,Of course ads are what you would expect from a...,1
9302,Love it! Amazon is just about the place to get...,2


In [268]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_size = 'right'

In [269]:
max_length = 512

def tokenizerFunction(string):
  # Tokenize a list of texts and return a dictionary with the tokenization outputs
    tokens = tokenizer(string, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
    return tokens 

train_encodings = train_data['content'].apply(tokenizerFunction)
val_encodings = val_data['content'].apply(tokenizerFunction)

In [270]:
train_encodings

0        [input_ids, attention_mask]
1        [input_ids, attention_mask]
2        [input_ids, attention_mask]
3        [input_ids, attention_mask]
4        [input_ids, attention_mask]
                    ...             
20901    [input_ids, attention_mask]
20902    [input_ids, attention_mask]
20903    [input_ids, attention_mask]
20904    [input_ids, attention_mask]
20905    [input_ids, attention_mask]
Name: content, Length: 20906, dtype: object

In [271]:
val_encodings

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
9299    [input_ids, attention_mask]
9300    [input_ids, attention_mask]
9301    [input_ids, attention_mask]
9302    [input_ids, attention_mask]
9303    [input_ids, attention_mask]
Name: content, Length: 9304, dtype: object

In [272]:
def creatingDataframe(encodings):
  indices = encodings.index.to_numpy()

  df = pd.DataFrame(columns=['input_ids', 'attention_mask'])
  
  # Assuming 'content' is a Series or List containing tuples/lists
  for element in encodings:
    input_ids = element['input_ids'][0].numpy()              # Replace with correct index based on your data structure
    attention_mask = element['attention_mask'][0].numpy()    # Replace with correct index
    df.loc[len(df.index)] = [input_ids, attention_mask]
  df_index = pd.DataFrame(indices, columns=['index'])
  df['index'] = df_index['index']
  return df

# Batch Size

In [273]:
# Define batch size and create data loaders
batch_size = 16  # Adjust as needed

In [274]:
# Create datasets
# Convert labels to tensors
train_dataset = creatingDataframe(train_encodings)
train_labels = train_data['sentiment'].reset_index(name='label')
train_dataset = pd.merge(train_dataset, train_labels, on='index')

val_dataset = creatingDataframe(val_encodings)
val_labels = val_data['sentiment'].reset_index(name='label')
val_dataset = pd.merge(val_dataset, val_labels, on='index')

# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_data_collator)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=default_data_collator)

In [275]:
train_dataset = train_dataset.drop('index', axis=1)
val_dataset = val_dataset.drop('index', axis=1)

In [276]:
train_dataset

,input_ids,attention_mask,label
0,"[94, 3621, 6, 68, 38, 27, 183, 14139, 494, 27,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[1804, 917, 68, 8, 12347, 960, 744, 31, 17, 16...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2276, 19, 230, 3, 5828, 50, 9559, 12, 8, 500,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,"[27, 43, 3, 7483, 633, 1245, 1173, 1, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",2
4,"[2181, 715, 27, 805, 45, 2536, 34, 373, 54, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
...,...,...,...
20901,"[1182, 2684, 50, 9559, 1, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
20902,"[1237, 6097, 2309, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
20903,"[12320, 652, 3566, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
20904,"[12, 63, 32, 17, 9, 142, 14460, 23, 9, 1, 0, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",2


In [277]:
val_dataset

,input_ids,attention_mask,label
0,"[94, 3621, 6, 68, 38, 27, 183, 14139, 494, 27,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[1804, 917, 68, 8, 12347, 960, 744, 31, 17, 16...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2276, 19, 230, 3, 5828, 50, 9559, 12, 8, 500,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,"[27, 43, 3, 7483, 633, 1245, 1173, 1, 0, 0, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",2
4,"[2181, 715, 27, 805, 45, 2536, 34, 373, 54, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
...,...,...,...
9299,"[12504, 12, 36, 1006, 6, 68, 141, 2225, 323, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
9300,"[101, 2997, 333, 2536, 55, 27, 12855, 43, 435,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
9301,"[1129, 503, 6543, 33, 125, 25, 133, 1672, 45, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
9302,"[2129, 34, 55, 2536, 19, 131, 81, 8, 286, 12, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2


In [278]:
# Define a custom dataset class
#class CustomDataset(Dataset):
#    def __init__(self, dataframe, data):
#        self._data = data
#
#    def __len__(self):
#        return len(self._data)
#
#    def __getitem__(self, idx):
#        label = self.labels[idx]
#        input_ids = self.input_ids[idx]
#        attention_mask = self.attention_masks[idx]
#        return {
#            'input_ids': torch.tensor(dataframe['input_ids'], dtype=torch.long),
#            'attention_mask': torch.tensor(dataframe['attention_mask'], dtype=torch.long),
#            'labels': torch.tensor(dataframe['label'], dtype=torch.long)
#        }

In [279]:
#train_dataset = CustomDataset(train_dataset, train_data)
#val_dataset = CustomDataset(val_dataset, val_data)

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

In [280]:
path = ''

In [281]:
# Define training arguments with early stopping
training_args = TrainingArguments(
    # Positional arguments:
    output_dir= path+"./t5-small_wt_sentiment_model",
    logging_dir= path+"./logs_t5s_wt",
    #device=device,

    # Keyword arguments:
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    #eval_steps=250,                         # Adjust as needed
    save_total_limit=2,                     # Adjust as needed. Only last 2 models are saved. Older ones are deleted.
    num_train_epochs=10,                     # Adjust as needed
    save_steps=500,                         # Adjust as needed
    metric_for_best_model="eval_loss",      # Use validation loss for early stopping
    learning_rate= 7e-05,
    warmup_steps=500,
    weight_decay=0.001,
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [282]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):

    logits = eval_pred[0]
    logits, labels = logits
    
    labels = eval_pred[1]
    
    # Convert logits to predicted labels
    predicted_labels = np.argmax(logits, axis=1)

    metrics = {}
    # Calculate accuracy
    accuracy = accuracy_score(labels, predicted_labels)
    balanced_accuracy = balanced_accuracy_score(labels, predicted_labels)
    metrics['accuracy'] = accuracy
    metrics['balanced_accuracy'] = balanced_accuracy
    
    # Calculate precision, recall, F1 score, and support for each class
    precision, recall, f1, support = precision_recall_fscore_support(labels, predicted_labels)    
    

    # Create a dictionary to store metrics for each class
    class_metrics = {}
    for i in range(len(precision)):
        class_metrics[f'class_{i}'] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1': f1[i],
            'support': support[i]
        }

    precision_ALL = precision_score(labels, predicted_labels, average='macro')
    recall_ALL = recall_score(labels, predicted_labels, average='macro')
    f1_ALL = f1_score(labels, predicted_labels, average='macro')

    metrics['precision'] = precision_ALL
    metrics['recall'] = recall_ALL
    metrics['f1'] = f1_ALL
    
    # Print and return the metrics
    for class_name, c_metrics in class_metrics.items():
        metrics[f'{class_name}_precision'] = c_metrics["precision"]
        metrics[f'{class_name}_recall'] = c_metrics["recall"]
        metrics[f'{class_name}_f1'] = c_metrics["f1"]
        metrics[f'{class_name}_support'] = c_metrics["support"]

    return metrics

In [283]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [284]:
# Fine-tune the model
trainer.train()

/home/patrick.araujo/miniconda3/envs/llama/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 6.50 GiB. GPU 0 has a total capacity of 23.68 GiB of which 6.43 GiB is free. Including non-PyTorch memory, this process has 17.24 GiB memory in use. Of the allocated memory 15.19 GiB is allocated by PyTorch, and 900.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Use the trainer to predict labels on the validation dataset
predictions = trainer.predict(val_dataset)
logits = predictions[0]
predictions, labels = logits
predicted_labels = np.argmax(predictions, axis=1)
# Extract the true labels from the validation dataset
true_labels = val_dataset['label']

# Generate the confusion matrix
unique_labels = dataset['sentiment'].unique()
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=unique_labels)

In [ ]:
import matplotlib.pyplot as plt
# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap=plt.cm.Blues)

# Save the plot as an image file (e.g., PNG)
plt.savefig('confusion_matrix_t5wt.png')
plt.savefig('confusion_matrix_t5wt.svg')

# Optionally, display the plot
plt.show()

In [ ]:
# Evaluate the model
results = trainer.evaluate()

In [ ]:
results

In [ ]:
output_file_path = './t5-small_wt_sentiment_model'
destination = path+output_file_path

# Save the model
model.save_pretrained(destination)